# Schedule generation

Now that we have simulation data on the interactions between sensors and mules, we want to generate an upload schedule based on sampling frequency and batched updates.

We will collect the following thingomaboobers:

`schedule.csv`

| sensor_id | mule_id | sample_time | pickup_time | batch_time | data_length |
|:--:|:--:|:--:|:--:|:--:|:--:|
| i = 0, ..., 999 | j = 0, ..., 99 | seconds | seconds | seconds | bytes |

where `sample_time` is the time that the sensor generated the sample, `pickup_time` is the time that the mule walked into the sensor's range and picked up the packet (which is also the upload time for a non-privacy-preserving setup), and `batch_time` is the time that the packet would be uploaded if it the upload time is delayed and uploads are batched into constant sized chunks for privacy reasons. 

We will also have the following parameters that can be twiggled:

- `number of mules` - number (integer) of mules that are included in the simulation
- `number of sensors` - number (integer) of sensors that are included in the simulation
- `advertisement period` - time (in seconds) between each sensor's BLE advertisement used to discover nearby mules
- `connection time` - time (in seconds) needed for a connection to form before data can be transferred
- `ble throughput` - rate (in bytes per second) of data transfer from a sensor to a mule
- `sample period` - time (in seconds) between each sample that a sensor takes
- `sample length` - size (in bytes) of the samples a sensor transfers to a mule
- `batch period mean` - average time (in seconds) between each batch the mule uploads
- `batch period std` - deviation in time (in seconds) between each batch the mule uploads 
- `batch period min` - minimum time (in seconds) between each batch the mule uploads
- `batch length` - size (in bytes) of the batches a mule uploads to the cloud

I think this is all we need for now to evaluate the baseline and Express. We assume that each sensor connects to only one mule at a time, while a mule can connect to an arbitrary number of sensors at a time. After a sensor connects to a mule, it stays connected and transfers data as long as the mule is within range. Once the mule leaves the sensor's range, the sensor immediately starts advertisements and looks for a new mule to form a connection with.

## To generate new data using this notebook

Either run this notebook and input different parameters into the `generate_schedule` function, or import the function into another Python script to use it in your own code. This should suffice for basic trials.

Currently, we assume that all sensors have the same advertisement period, connection time, BLE throughput, sample period, and sample length, and that all mules have the same batch length. To make these values differ across sensors and mules or to have them change over time, you will have to make some small changes to the code for value generation and proper bookkeeping. This should be reasonably straightforward.

We also assume that each mule can connect to an arbitrary number of sensors and store an arbitrary amount of data collected from sensors. This code does not support any connection or memory restrictions on the mule's side. If you want to consider the case where each mule can only connect to one sensor at a time or only accepts up to, say, 10KB of data at any time and shuts down all BLE connections after hitting that limit, you will have to make major adjustments to how the schedule is generated, and potentially rewrite substantial portions of the code. This is because we currently generate all traffic outward from sensors to mules (unaware of the mules' internal states) before retroactively batching all of the uploads for each mule. This allows us to consider each sensor's outward traffic independent of all other sensors. However, a connection or memory limit for mules would conflate the traffic from multiple sensors sending packets to the same mule, preventing us from considering each sensor individually.

In [1]:
# Import libraries.
import numpy as np
import pandas as pd
import math

# Set a seed for our stochastic uploads 
np.random.seed(1337)

# Toss everything into a function for looping purposes
def generate_schedule(num_mules = 100, # integer <= 100
                      num_sensors = 1000, # integer <= 1000
                      advertisement_period = 2.0, # seconds
                      connection_time = 1.5, # seconds
                      ble_throughput = 125000.0, # bytes per second
                      sample_period = 10.0, # seconds
                      sample_length = 128, # bytes
                      batch_period_min = 60.0, # seconds
                      batch_period_max = 600.0, # seconds
                      batch_length = 100000, # bytes
                      interaction_file = 'prob_data/continual_motion/interactions.csv', # csv file path
                      save_file = 'prob_data/random_uploads/schedule.csv', # csv file path
                      verbose = False
                     ):
    
    if verbose:
        print('Reading interactions from `{}`.'.format(interaction_file))
        print('Using {} sensors and {} mules.'.format(num_sensors, num_mules))
        print('Mules upload every {} seconds on average.'.format((batch_period_min + batch_period_max)/2.0))
    
    # Read in data files and downsample as necessary.
    interaction_df = pd.read_csv(interaction_file)
    smol_interactions = interaction_df.loc[(interaction_df['sensor_id'] < num_sensors) & 
                                           (interaction_df['mule_id'] < num_mules)]
    # Sort the interactions by sample ID and interaction time.
    smol_interactions = smol_interactions.sort_values(['sensor_id', 'interaction_time'])

    # Calculate some useful numbers.
    time_per_sample = sample_length / ble_throughput 
    samples_per_batch = math.floor(batch_length / sample_length)

    # Start bookkeeping.
    cur_sensor = 0 # Keeps track of the sensor we are generating a schedule for.
    next_sample = 0 # Accumulates samples for each sensor to send.
    cur_end_time = 0.0 # Keeps track of the latest action time for each sensor.
    schedule = [] # Records our resulting upload schedule.
    
    
    if verbose:
        print('Generating sensor-to-mule data transfers...')
    
    # Iterate through each row of sensor-mule interactions.
    for index, row in smol_interactions.iterrows():
        # Grab the sensor and mule used in this interaction.
        new_sensor = int(row['sensor_id'])
        new_mule = int(row['mule_id'])

        # If we moved on to the next sensor, reset our bookkeeping.
        if cur_sensor != new_sensor:
            cur_sensor = new_sensor
            next_sample = 0
            cur_end_time = 0.0

        # If the sensor has already taken actions beyond this time, ignore this row.
        new_end_time = row['interaction_time'] + row['interaction_duration']
        if cur_end_time >= new_end_time:
            continue

        # Otherwise, we advertise and attempt to start a connection.
        time_passed = max(0.0, row['interaction_time'] - cur_end_time)
        new_start_time = cur_end_time + math.ceil(time_passed / advertisement_period) * advertisement_period
        # If there is not enough time for a connection, we waste some time and move on with our lives.
        cur_end_time = new_start_time + connection_time
        if cur_end_time >= new_end_time:
            cur_end_time = new_end_time
            continue
        # Otherwise, we successfully connected to the mule and have time to do stuff.

        # The first thing we do is dump a bunch of accumulated samples onto the mule as fast as possible
        # as long as the mule is connected.
        while next_sample * sample_period <= cur_end_time and cur_end_time + time_per_sample <= new_end_time:
            cur_end_time += time_per_sample
            # Send the sample with placeholders for batching.
            schedule.append([new_sensor,                  # sensor_id
                             new_mule,                    # mule_id 
                             next_sample * sample_period, # sample_time
                             cur_end_time,                # pickup_time
                             -1,                          # batch_time
                             sample_length])              # data_length
            next_sample += 1

        # After we have done that, we continue the connection and send new samples as they come in.
        while next_sample * sample_period + time_per_sample <= new_end_time:
            # Send the sample with placeholders for batching.
            schedule.append([new_sensor,                                    # sensor_id
                             new_mule,                                      # mule_id 
                             next_sample * sample_period,                   # sample_time
                             next_sample * sample_period + time_per_sample, # pickup_time
                             -1,                                            # batch_time
                             sample_length])                                # data_length
            next_sample += 1

        # Finally, once we are done sending all the samples that can be sent, we close out the interaction
        # and update the bookkeeping as necessary.
        cur_end_time = new_end_time

    
    if verbose:
        print('Generating mule batched upload times...')
    
    # Now we want to figure out which batches the mule will end up sending each sample in.
    # We do this based on chronological order of sample receipt. 
    labels = ['sensor_id', 'mule_id', 'sample_time', 'pickup_time', 'batch_time', 'data_length']
    schedule = pd.DataFrame(schedule, columns=labels)
    schedule = schedule.sort_values(['mule_id', 'pickup_time'])

    # Setup some bookkeeping.
    cur_mule = 0
    next_batch_time = 0.0
    next_batch_length = 0

    # Iterate through each row of sensor-mule packet transfers.
    for index, row in schedule.iterrows():
        # Grab some useful values.
        new_mule = int(row['mule_id'])
        pickup_time = row['pickup_time']

        # If we are looking at a new mule, reset our bookkeeping.
        if cur_mule != new_mule:
            cur_mule = new_mule
            next_batch_time = 0.0
            next_batch_length = 0

        # If we already sent the next batch, increment our batch index.
        while next_batch_time <= pickup_time:
            next_batch_time += np.random.uniform(batch_period_min, batch_period_max)
            next_batch_length = 0

        # Increment the number of samples in this upcoming batch. If there are too many, we postpone
        # this sample until the next batch.
        next_batch_length += 1
        if next_batch_length > samples_per_batch:
            next_batch_time += np.random.uniform(batch_period_min, batch_period_max)
            next_batch_length = 1

        # Update the batch upload time.
        schedule.at[index, 'batch_time'] = next_batch_time
        
        
    # Save our results
    with open(save_file, 'w') as f:
        # Record parameters.
        f.write('num_mules,num_sensors,advertisement_period,connection_time,ble_throughput,sample_period,sample_length,batch_period_min,batch_period_max,batch_length\n')
        f.write('{},{},{},{},{},{},{},{},{}\n\n'.format(num_mules,num_sensors,advertisement_period,connection_time,ble_throughput,sample_period,sample_length,batch_period_min,batch_period_max,batch_length))

        # Record schedule.
        schedule.to_csv(f, index=False)

        
    # Quick printout for funsies.
    if verbose:
        print('Saved results to {}.'.format(save_file))
        print(schedule.head())

In [2]:
# Try out our function
generate_schedule(batch_period_max=300.0, verbose=True)

Reading interactions from `prob_data/continual_motion/interactions.csv`.
Using 1000 sensors and 100 mules.
Mules upload every 180.0 seconds on average.
Generating sensor-to-mule data transfers...
Generating mule batched upload times...
Saved results to prob_data/random_uploads/schedule.csv.
        sensor_id  mule_id  sample_time  pickup_time  batch_time  data_length
184214        697        0          0.0     1.501024         122          128
261649        977        0          0.0     1.501024         122          128
166797        624        0          0.0     3.501024         122          128
170615        638        0          0.0     7.501024         122          128
3905           17        0         20.0    20.001024         122          128


## Generate schedules for different numbers of mules

Now that the schedule generator is working, let's spit out a few traces for variable number of mules.

In [3]:
# Set parameters.
num_mules_list = [10, 20, 30, 40, 50, 60, 70 ,80, 90, 100] # integer <= 100

# Set save file.
save_file = 'prob_data/random_uploads/vary_mules/{}_mule_schedule.csv'

In [4]:
for num_mules in num_mules_list:
    generate_schedule(num_mules=num_mules, save_file=save_file.format(num_mules))
    print("Finished generating schedule for {} mules".format(num_mules))


Finished generating schedule for 10 mules
Finished generating schedule for 20 mules
Finished generating schedule for 30 mules
Finished generating schedule for 40 mules
Finished generating schedule for 50 mules
Finished generating schedule for 60 mules
Finished generating schedule for 70 mules
Finished generating schedule for 80 mules
Finished generating schedule for 90 mules
Finished generating schedule for 100 mules
